# Testing out Linear Programming

In [11]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [12]:
path = "../data/parts/"

In [13]:
# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data.csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data.csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("../data/games/top300.csv")

# Return original value if conversion fails
def safe_to_numeric(val):
    try:
        return pd.to_numeric(val)
    except (ValueError, TypeError):
        return val  

# Convert numeric columns from string to numeric types
cpus = cpus.apply(safe_to_numeric)
gpus = gpus.apply(safe_to_numeric)
rams = rams.apply(safe_to_numeric)
cases = cases.apply(safe_to_numeric)
motherboards = motherboards.apply(safe_to_numeric)
psus = psus.apply(safe_to_numeric)
storage = storage.apply(safe_to_numeric)
m2s = storage[storage['Storage Type'] == 'M.2 SSD']
satas = storage[storage['Storage Type'] == 'SATA SSD'] #SATA HDD was ignored to due niche use-cases
games = games.apply(safe_to_numeric)

# Normalize CPU & GPU Scores
def normalize_column(df, column_name):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    df[column_name + "_Normalized"] = (df[column_name] - min_val) / (max_val - min_val)
    return df

cpus = normalize_column(cpus, "Score")
gpus = normalize_column(gpus, "Score")

rams_min = rams["Capacity (GB)"].min()
rams_max = rams["Capacity (GB)"].max()
rams["Capacity (GB)_Normalized"] = 0.01 + 0.99 * (rams["Capacity (GB)"] - rams_min) / (rams_max - rams_min)

In [14]:
# selected_game = 'No Man's Sky'
# selected_game = 'ELDEN RING'
# selected_game = 'Red Dead Redemption 2'
# selected_game = 'Cyberpunk 2077'
selected_game = 'Counter-Strike 2'

# budget = 1000
budget = 1600
# budget = 2000
# budget = 2500
# budget = 3000

# Storage stuff
min_storage = 1024 #gb
min_m2 = 1
min_sata = 1

use_sata = True
ram_16 = True

# Select the game (copy a game name from top300.csv)
game_data = games[games['name'] == selected_game].iloc[0]

display(game_data)

app_id                                                           730
name                                                Counter-Strike 2
recommendations                                              4248820
genres                                    ['Action', 'Free To Play']
tags               ['FPS', 'Shooter', 'Multiplayer', 'Competitive...
CPU                                                             1300
GPU                                                             4500
memory                                                           8.0
Name: 0, dtype: object

In [15]:
''' ---Filtering--- '''
# Available brands for each part
# CPU: AMD, Intel
# GPU: AMD, ASRock, Asus, EVGA, Gainward, Gigabyte, Intel, MSI, Nvidia, PNY, PowerColor, Sapphire, Zotac
# Ram : Kingston, XPG, Fanxiang, PNY, Corsair, Samsung, Apacer, KLEVV
# Motherboard (Mobo): ASUS, MSI, ASRock, GIGABYTE, Colorful, NZXT
# PSU: 1st Player, Asus, Cooler, Deepcool, Gigabyte, MSI, Silverstone, Thermaltake
# Case: Armaggeddon, 1st Player, Lian Li

# Define allowed brands (customize as needed)
allowed_cpu_brands = ["Intel"]
allowed_gpu_brands = ["Nvidia", "Asus", "Gainward"]
allowed_ram_brands = ["Kingston", "Corsair"]
allowed_psu_brands = ["1st Player", "ASRock"]
allowed_mobo_brands = ["MSI", "ASUS"]
allowed_case_brands = ["1st Player"]
allowed_storage_brands = ["WD"]

# Filter each DataFrame by brand
cpus_filtered = cpus[cpus["Brand"].isin(allowed_cpu_brands)]
gpus_filtered = gpus[gpus["Brand"].isin(allowed_gpu_brands)]
rams_filtered = rams[rams["Brand"].isin(allowed_ram_brands)]
psus_filtered = psus[psus["Brand"].isin(allowed_psu_brands)]
motherboards_filtered = motherboards[motherboards["Brand"].isin(allowed_mobo_brands)]
cases_filtered = cases[cases["Brand"].isin(allowed_case_brands)]
m2s_filtered = m2s[m2s["Brand"].isin(allowed_storage_brands)]
satas_filtered = satas[satas["Brand"].isin(allowed_storage_brands)]

In [16]:
# stuff to make it easy to pick between filtered and non filtered parts
# and to force the min ram to be 16 gb since that is the standard for having a pc

filter = False


if filter == True:
    cpusf = cpus_filtered.reset_index(drop=True)
    gpusf = gpus_filtered.reset_index(drop=True)
    ramsf = rams_filtered.reset_index(drop=True)
    psusf = psus_filtered.reset_index(drop=True)
    mobosf = motherboards_filtered.reset_index(drop=True)
    casesf = cases_filtered.reset_index(drop=True)
    m2sf = m2s_filtered.reset_index(drop=True)
    satasf = satas_filtered.reset_index(drop=True)

    lenCPU = range(len(cpusf))
    lenGPU = range(len(gpusf))
    lenRAM = range(len(ramsf))
    lenPSU = range(len(psusf))
    lenMOBO = range(len(mobosf))
    lenCase = range(len(casesf))
    lenM2 = range(len(m2sf))
    lenSATA = range(len(satasf))

else:
    cpusf = cpus.reset_index(drop=True)
    gpusf = gpus.reset_index(drop=True)
    ramsf = rams.reset_index(drop=True)
    psusf = psus.reset_index(drop=True)
    mobosf = motherboards.reset_index(drop=True)
    casesf = cases.reset_index(drop=True)
    m2sf = m2s.reset_index(drop=True)
    satasf = satas.reset_index(drop=True)

    lenCPU = range(len(cpusf))
    lenGPU = range(len(gpusf))
    lenRAM = range(len(ramsf))
    lenPSU = range(len(psusf))
    lenMOBO = range(len(mobosf))
    lenCase = range(len(casesf))
    lenM2 = range(len(m2sf))
    lenSATA = range(len(satasf))

# Validation to ensure filtered lists are not empty
if cpus.empty or gpus.empty or rams.empty or psus.empty or motherboards.empty or cases.empty:
    raise ValueError("One or more component categories have no items after filtering. Adjust brand filters.")
if mobosf['NVMe Slot'].max() < min_m2:
    raise ValueError(f"min_m2 ({min_m2}) is greater than max NVMe slots ({mobosf['NVMe Slot'].max()})")
if min_storage > mobosf['NVMe Slot'].max() * m2sf['Capacity (GB)'].max():
    raise ValueError("Impossible: min_storage is too high for available mobos and SSDs.")

### Algoritm B (NEW)

In [17]:
''' Create compatibility dictionary for MOBO'''
# Socket
mobo_cpu_compatibility_dict = {
    (i, j): int(cpu['Socket'] == mobo['Socket'])
    for i, cpu in cpusf.iterrows()
    for j, mobo in mobosf.iterrows()
}

# Size (for Case)
mobo_case_compatibility_dict = {
    (i, j): int(case['Size'] == mobo['Size'])
    for i, case in casesf.iterrows()
    for j, mobo in mobosf.iterrows()
}

# DDR 
mobo_ram_compatibility_dict = {
    (i, j): int(ram['DDR'] == mobo['DDR'])
    for i, ram in ramsf.iterrows()
    for j, mobo in mobosf.iterrows()
}

In [21]:
# Use leading zeros to name LP Variables
# This is because, LP Variables are sorted lexicographically (0, 1, 10, 2, 3, etc.)
def add_leading_zeros(i, width=4):
    return str(i).zfill(width)

# --- Creating LP Problem --- 
problem = LpProblem("Desktop_Optimization", LpMaximize)

# --- Define PC Components ---
cpu_vars = [LpVariable(f"cpu_{add_leading_zeros(i)}", cat="Binary") for i in lenCPU]
gpu_vars = [LpVariable(f"gpu_{add_leading_zeros(i)}", cat="Binary") for i in lenGPU]
psu_vars = [LpVariable(f"psu_{add_leading_zeros(i)}", cat="Binary") for i in lenPSU]
mobo_vars = [LpVariable(f"mb_{add_leading_zeros(i)}", cat="Binary") for i in lenMOBO]
case_vars = [LpVariable(f"case_{add_leading_zeros(i)}", cat="Binary") for i in lenCase]
ram_vars = [LpVariable(f"ram_{add_leading_zeros(i)}", cat="Binary") for i in lenRAM]
ram_count = LpVariable("ram_count", lowBound=1, upBound=mobosf['RAM Slot'].max(), cat="Integer")
ram_count_selected = [LpVariable(f"ram_count_selected_{add_leading_zeros(i)}", lowBound=0, upBound=mobosf['RAM Slot'].max(), cat="Integer") for i in lenRAM] #Auxiliary Variable

m2_vars = [LpVariable(f"m2_{add_leading_zeros(i)}", cat="Binary") for i in lenM2]
m2_count = LpVariable("m2_count", lowBound=1, upBound=mobosf['NVMe Slot'].max(), cat="Integer")
m2_count_selected = [LpVariable(f"m2_count_selected_{add_leading_zeros(i)}", lowBound=0, upBound=mobosf['NVMe Slot'].max(), cat="Integer") for i in lenM2] #Auxiliary Variable

sata_vars = [LpVariable(f"sata_{add_leading_zeros(i)}", cat="Binary") for i in lenSATA]
sata_vars_test = [LpVariable(f"sata_{i}", cat="Binary") for i in lenSATA]
sata_count = LpVariable("sata_count", lowBound=1, upBound=mobosf['SATA Slot'].max(), cat="Integer")
sata_count_selected = [LpVariable(f"sata_count_selected_{add_leading_zeros(i)}", lowBound=0, upBound=mobosf['SATA Slot'].max(), cat="Integer") for i in lenSATA] #Auxiliary Variable


''' ---Part Count Constraint--- '''
# Only select exactly one of each major part
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(ram_vars) == 1, "Select_One_RAM"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"
problem += lpSum(mobo_vars) == 1, "Select_One_MOBO"
problem += lpSum(case_vars) == 1, "Select_One_Case"

''' ---GPU Wattage Constraints--- '''
# GPU x Power Supply
problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Recommended Power"] for i in lenGPU) <=
    lpSum(psu_vars[i] * psusf.iloc[i]["Wattage"] for i in lenPSU),
    "PSU_Power_Constraint",
)

''' Storage Constraints '''
problem += (
    lpSum(m2_count_selected[i] * m2sf.iloc[i]["Capacity (GB)"] for i in lenM2) +
    lpSum(sata_count_selected[i] * satasf.iloc[i]["Capacity (GB)"] for i in lenSATA)
    >= min_storage,
    "Minimum_Total_Storage"
)

''' Default: 1 M.2 SSD 512GB Storage '''
''' M.2 SSD NVMe '''
problem += m2_count >= min_m2, "Minimum_M2"
problem += m2_count <= lpSum(mobo_vars[i] * mobosf.iloc[i]['NVMe Slot'] for i in lenMOBO)

for i in lenM2:
    problem += m2_count_selected[i] <= m2_count
    problem += m2_count_selected[i] <= mobosf['NVMe Slot'].max() * m2_vars[i]

problem += lpSum(m2_count_selected[i] for i in lenM2) == m2_count

''' SATA SSD '''
if use_sata:
    problem += sata_count >= min_sata, "Minimum_SATA"
    problem += sata_count <= lpSum(mobo_vars[i] * mobosf.iloc[i]['SATA Slot'] for i in lenMOBO)

    for i in lenSATA:
        problem += sata_count_selected[i] <= sata_count
        problem += sata_count_selected[i] <= mobosf['SATA Slot'].max() * sata_vars[i]
    
    problem += lpSum(sata_count_selected[i] for i in lenSATA) == sata_count
else:
    for i in lenSATA:
        problem += sata_vars[i] == 0
        problem += sata_count_selected[i] == 0

''' ---Motherboard Constraints--- '''
# Socket 
for i in lenCPU:
    problem += (
        cpu_vars[i] <= lpSum(mobo_cpu_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"CPU_Socket_Compatibility_{add_leading_zeros(i)}"
    )

# Case
for i in lenCase:
    problem += (
        case_vars[i] <= lpSum(mobo_case_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"Case_Size_Compatibility_{add_leading_zeros(i)}"
    )

# DDR
for i in lenRAM:
    problem += (
        ram_vars[i] <= lpSum(mobo_ram_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"RAM_DDR_Compatibility_{add_leading_zeros(i)}"
    )

# RAM Count
problem += ram_count >= 1, "At_Least_One_RAM"
problem += ram_count <= lpSum(mobo_vars[i] * mobosf.iloc[i]["RAM Slot"] for i in lenMOBO)

''' ---RAM Selection Algorithm--- '''
# For each RAM Model, link count to total count
for i in lenRAM:
    problem += ram_count_selected[i] <= ram_count
    problem += ram_count_selected[i] <= 4 * ram_vars[i]
    problem += ram_count_selected[i] >= ram_count - (1 - ram_vars[i]) * 4
    problem += ram_count_selected[i] >= 0

''' ---Game Constraints--- '''
problem += (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Score"] for i in lenCPU) >= game_data["CPU"],
    "Game_CPU_Constraint"
)

problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Score"] for i in lenGPU) >= game_data["GPU"],
    "Game_GPU_Constraint"
)

memory_requirement = int(game_data["memory"])
if (ram_16 and memory_requirement <= 16): # If minimum RAM is set to 16GB
    memory_requirement = 16

problem += (
    lpSum(ram_count_selected[i] * ramsf.iloc[i]['Capacity (GB)'] for i in lenRAM) >= memory_requirement,
    "Game_Memory_Constraint"
)

''' ---Cost Function--- '''
total_cost = (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Price"] for i in lenCPU) +
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Price"] for i in lenGPU) +
    lpSum(ram_count_selected[i] * ramsf.iloc[i]["Price"] for i in lenRAM) +
    lpSum(psu_vars[i] * psusf.iloc[i]["Price"] for i in lenPSU) +
    lpSum(mobo_vars[i] * mobosf.iloc[i]["Price"] for i in lenMOBO) +
    lpSum(case_vars[i] * casesf.iloc[i]["Price"] for i in lenCase) +
    lpSum(m2_count_selected[i] * m2sf.iloc[i]["Price"] for i in lenM2) +
    lpSum(sata_count_selected[i] * satasf.iloc[i]["Price"] for i in lenSATA)
)

problem += total_cost <= budget, "Budget_Constraint"

''' ---Performance Function---'''
# Parameters
cpu_weight = 0.1
gpu_weight = 0.895
ram_weight = 0.005
dual_channel_bonus_value = 0.02

# Extra variables
dual_channel_bonus_var = LpVariable("dual_channel_bonus_var", cat="Binary")

# Function
total_performance = (
    cpu_weight * lpSum(cpu_vars[i] * cpusf.iloc[i]["Score_Normalized"] for i in lenCPU) +
    gpu_weight * lpSum(gpu_vars[i] * gpusf.iloc[i]["Score_Normalized"] for i in lenGPU) +
    ram_weight * lpSum(ram_count_selected[i] * ramsf.iloc[i]["Capacity (GB)_Normalized"] for i in lenRAM) +
    dual_channel_bonus_value * dual_channel_bonus_var
)

'''
Dual-Channel Activation TLDR:
Solver has to toggle on-off the binary "dual_channel_bonus_var" to figure out
optimal ram selection based on budget & game.
'''
problem += total_performance
problem += ram_count_selected >= 2 * dual_channel_bonus_var, "Dual-Channel Activation Condition"

# --- Solve ---
problem.solve()

1

In [22]:
# Example on the importance of leading zeros for index referencing
print(lpSum(sata_vars[i] for i in lenSATA))
print(lpSum(sata_vars_test[i] for i in lenSATA))

sata_0000 + sata_0001 + sata_0002 + sata_0003 + sata_0004 + sata_0005 + sata_0006 + sata_0007 + sata_0008 + sata_0009 + sata_0010
sata_0 + sata_1 + sata_10 + sata_2 + sata_3 + sata_4 + sata_5 + sata_6 + sata_7 + sata_8 + sata_9


### Output

In [ ]:
# --- Output Results ---

# Get selected components
selected_cpu_idx = [i for i in lenCPU if cpu_vars[i].value() == 1][0]
selected_gpu_idx = [i for i in lenGPU if gpu_vars[i].value() == 1][0]
selected_psu_idx = [i for i in lenPSU if psu_vars[i].value() == 1][0]
selected_mobo_idx = [i for i in lenMOBO if mobo_vars[i].value() == 1][0]
selected_case_idx = [i for i in lenCase if case_vars[i].value() == 1][0]

selected_ram_idx = [i for i in lenRAM if ram_count_selected[i].value() > 0][0]
selected_ram_count = ram_count_selected[selected_ram_idx].value()
selected_ram_model = ramsf.iloc[selected_ram_idx]['Name']
selected_ram_brand = ramsf.iloc[selected_ram_idx]['Brand']

# M.2 SSDs: Get selected models and their counts
selected_m2 = [
    (i, m2sf.iloc[i], int(m2_count_selected[i].value()))
    for i in range(len(m2sf))
    if m2_count_selected[i].value() > 0
]

# SATA SSDs: Get selected models and their counts
selected_sata = [
    (i, satasf.iloc[i], int(sata_count_selected[i].value()))
    for i in range(len(satasf))
    if sata_count_selected[i].value() > 0
]

# Part names
selected_cpu = cpusf.iloc[selected_cpu_idx]['Name']
selected_gpu = gpusf.iloc[selected_gpu_idx]['Name']
selected_psu = psusf.iloc[selected_psu_idx]['Name']
selected_mobo = mobosf.iloc[selected_mobo_idx]['Name']
selected_case = casesf.iloc[selected_case_idx]['Name']

# Scores
selected_cpu_score = cpusf.iloc[selected_cpu_idx]["Score"]
selected_gpu_score = gpusf.iloc[selected_gpu_idx]["Score"]

# Prices
cpu_price = cpusf.iloc[selected_cpu_idx]["Price"]
gpu_price = gpusf.iloc[selected_gpu_idx]["Price"]
psu_price = psusf.iloc[selected_psu_idx]["Price"]
mobo_price = mobosf.iloc[selected_mobo_idx]["Price"]
case_price = casesf.iloc[selected_case_idx]["Price"]
ram_unit_price = ramsf.iloc[selected_ram_idx]["Price"]
ram_price = ram_unit_price * selected_ram_count

# Storage prices
total_m2_price = sum(
    m2sf.iloc[i]["Price"] * int(m2_count_selected[i].value())
    for i in range(len(m2sf))
    if m2_count_selected[i].value() > 0
)
total_sata_price = sum(
    satasf.iloc[i]["Price"] * int(sata_count_selected[i].value())
    for i in range(len(satasf))
    if sata_count_selected[i].value() > 0
)
total_storage_price = total_m2_price + total_sata_price

# Calculate Total RAM capacity
total_ram_capacity = selected_ram_count * ramsf.iloc[selected_ram_idx]["Capacity (GB)"]

# Total cost manually
total_cost_manual = (
    cpu_price + gpu_price + ram_price + psu_price + mobo_price + case_price + total_storage_price
)

# --- ✅ Output: Selected PC Build Summary ---

print("🎮 Recommended PC Build for:", selected_game)
print("-" * 50)
print(f"🧠 Required CPU Benchmark Score : {game_data['CPU']}")
print(f"🎮 Required GPU Benchmark Score : {game_data['GPU']}")
print(f"🧠 Required Memory              : {game_data['memory']} GB")
print("-" * 50)

# CPU
print(f"🧩 CPU: {selected_cpu}")
print(f"   └─ Price  : RM {cpu_price}")
print(f"   └─ Score  : {selected_cpu_score}")
print("")

# GPU
print(f"🖼️  GPU: {selected_gpu}")
print(f"   └─ Price  : RM {gpu_price}")
print(f"   └─ Score  : {selected_gpu_score}")
print("")

# RAM
print(f"🧠 RAM: {selected_ram_brand} - {selected_ram_model}")
print(f"   └─ Modules      : {int(selected_ram_count)}")
print(f"   └─ Capacity     : {total_ram_capacity} GB total")
print(f"   └─ Unit Price   : RM {ram_unit_price}")
print(f"   └─ Total Price  : RM {ram_price}")
print("")

# PSU
print(f"🔌 PSU: {selected_psu}")
print(f"   └─ Price: RM {psu_price}")
print("")

# Motherboard
print(f"🧩 Motherboard: {selected_mobo}")
print(f"   └─ Price: RM {mobo_price}")
print("")

# Case
print(f"🖥️  Case: {selected_case}")
print(f"   └─ Price: RM {case_price}")
print("")

# M.2 SSDs
print("💾 M.2 SSDs:")
if selected_m2:
    for idx, row, count in selected_m2:
        print(f"   └─ {row['Brand']} {row['Name']} x{count} "
              f"({row['Capacity (GB)']}GB, RM {row['Price']} each)")
else:
    print("   └─ None selected.")
print(f"   └─ Total M.2 Price: RM {total_m2_price}")
print("")

# SATA SSDs
print("💽 SATA SSDs:")
if selected_sata:
    for idx, row, count in selected_sata:
        print(f"   └─ {row['Brand']} {row['Name']} x{count} "
              f"({row['Capacity (GB)']}GB, RM {row['Price']} each)")
else:
    print("   └─ None selected.")
print(f"   └─ Total SATA Price: RM {total_sata_price}")
print("")

# Summary
print("=" * 50)
print(f"💰 Total Storage Cost : RM {total_storage_price}")
print(f"💸 Total Build Cost   : RM {total_cost_manual:.2f}")
print("=" * 50)
print("❗ Prices may vary depending on market fluctuations ❗")

🎮 Recommended PC Build for: Counter-Strike 2
--------------------------------------------------
🧠 Required CPU Benchmark Score : 1300
🎮 Required GPU Benchmark Score : 4500
🧠 Required Memory              : 8.0 GB
--------------------------------------------------
🧩 CPU: Ryzen 5 1600
   └─ Price  : RM 180
   └─ Score  : 12278

🖼️  GPU: Sapphire RX Vega 56 8GB Pulse
   └─ Price  : RM 500
   └─ Score  : 13193

🧠 RAM: Kingston - DDR4
   └─ Modules      : 2
   └─ Capacity     : 16.0 GB total
   └─ Unit Price   : RM 59
   └─ Total Price  : RM 118.0

🔌 PSU: PF650
   └─ Price: RM 169

🧩 Motherboard: A520M-A Pro
   └─ Price: RM 239

🖥️  Case: CS-111 V2
   └─ Price: RM 119

💾 M.2 SSDs:
   └─ WD SN350 x1 (512GB, RM 154 each)
   └─ Total M.2 Price: RM 154

💽 SATA SSDs:
   └─ ADATA SU650 x1 (512GB, RM 95 each)
   └─ Total SATA Price: RM 95

💰 Total Storage Cost : RM 249
💸 Total Build Cost   : RM 1574.00
❗ Prices may vary depending on market fluctuations ❗
